<a href="https://colab.research.google.com/github/aryanmangal769/stress_prediction_from_HRV/blob/main/Windowing_and_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install wfdb

In [ ]:
import os
import sys

# data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import seaborn as sns

# signal processing
from scipy import signal
from scipy.ndimage import label
from scipy.stats import zscore
from scipy.interpolate import interp1d
from scipy.integrate import trapz

# physionet data
import wfdb
from wfdb import processing

import numpy as np
import pandas as pd

In [ ]:
project_path = os.path.join(os.getcwd(), os.pardir)
data_path = os.path.join(project_path, '/content/drive/MyDrive/HRV/Github_Dataset/')
output_path = os.path.join(project_path, '/content/drive/MyDrive/HRV/Github_Dataset/output')



# Helper Functions

In [ ]:
def get_plot_ranges(start=10, end=20, n=5):
    '''
    Make an iterator that divides into n or n+1 ranges. 
    - if end-start is divisible by steps, return n ranges
    - if end-start is not divisible by steps, return n+1 ranges, where the last range is smaller and ends at n
    
    # Example:
    >> list(get_plot_ranges())
    >> [(0.0, 3.0), (3.0, 6.0), (6.0, 9.0)]

    '''
    distance = end - start
    for i in np.arange(start, end, np.floor(distance/n)):
        yield (int(i), int(np.minimum(end, np.floor(distance/n) + i)))

def group_peaks(p, threshold=5):
    '''
    The peak detection algorithm finds multiple peaks for each QRS complex. 
    Here we group collections of peaks that are very near (within threshold) and we take the median index 
    '''
    # initialize output
    output = np.empty(0)

    # label groups of sample that belong to the same peak
    peak_groups, num_groups = label(np.diff(p) < threshold)

    # iterate through groups and take the mean as peak index
    for i in np.unique(peak_groups)[1:]:
        peak_group = p[np.where(peak_groups == i)]
        output = np.append(output, np.median(peak_group))
    return output

def detect_peaks(ecg_signal, threshold=0.3, qrs_filter=None):
    '''
    Peak detection algorithm using cross corrrelation and threshold 
    '''
    if qrs_filter is None:
        # create default qrs filter
        t = np.linspace(1.5 * np.pi, 3.5 * np.pi, 15)
        qrs_filter = np.sin(t)
    
    # normalize data
    ecg_signal = (ecg_signal - ecg_signal.mean()) / ecg_signal.std()

    # calculate cross correlation
    similarity = np.correlate(ecg_signal, qrs_filter, mode="same")
    similarity = similarity / np.max(similarity)

    # return peaks (values in ms) using threshold
    return ecg_signal[similarity > threshold].index, similarity


def get_rr_corrected(df):
  # detect peaks
  peaks, similarity = detect_peaks(df.heartrate, threshold=0.3)

  # group peaks so we get a single peak per beat (hopefully)
  grouped_peaks = group_peaks(peaks)

  # RR-intervals are the differences between successive peaks
  rr = np.diff(grouped_peaks)

  rr_corrected = rr.copy()
  rr_corrected[np.abs(zscore(rr)) > 2] = np.median(rr)
  return rr_corrected

def plot_rr_corrected(rr_corrected,rr):

  plt.title("RR-intervals", fontsize=24)
  plt.xlabel("Time (ms)", fontsize=16)
  plt.ylabel("RR-interval (ms)", fontsize=16)

  plt.plot(rr, color="red", linewidth=1, label="RR-intervals")
  plt.plot(rr_corrected, color="green", linewidth=2, label="RR-intervals after correction")
  plt.legend(fontsize=20)

def timedomain(rr):

  results = {}
  
  hr = 60000/rr
  
  # mean RR-interval
  results['Mean RR (ms)'] = np.mean(rr)
  results['STD RR/SDNN (ms)'] = np.std(rr)
  results['Mean HR (Kubios\' style) (beats/min)'] = 60000/np.mean(rr)
  results['Mean HR (beats/min)'] = np.mean(hr)
  results['STD HR (beats/min)'] = np.std(hr)
  results['Min HR (beats/min)'] = np.min(hr)
  results['Max HR (beats/min)'] = np.max(hr)
  results['RMSSD (ms)'] = np.sqrt(np.mean(np.square(np.diff(rr))))
  results['NNxx'] = np.sum(np.abs(np.diff(rr)) > 50)*1
  results['pNNxx (%)'] = 100 * np.sum((np.abs(np.diff(rr)) > 50)*1) / len(rr)
  # print("Time domain metrics - automatically corrected RR-intervals:")
  # for k, v in results.items():
  #     print("- %s: %.2f" % (k, v))
  return results

def frequency_domain(rr, fs=4):

  # sample rate for interpolation
  steps = 1 / fs

  # create interpolation function based on the rr-samples. 
  x = np.cumsum(rr) / 1000.0
  f = interp1d(x, rr, kind='cubic',fill_value="extrapolate")

  # now we can sample from interpolation function
  xx = np.arange(1, np.max(x), steps)
  rr_interpolated = f(xx)
  # Estimate the spectral density using Welch's method
  fxx, pxx = signal.welch(x=rr_interpolated, fs=fs)
  
  '''
  Segement found frequencies in the bands 
    - Very Low Frequency (VLF): 0-0.04Hz 
    - Low Frequency (LF): 0.04-0.15Hz 
    - High Frequency (HF): 0.15-0.4Hz
  '''
  cond_vlf = (fxx >= 0) & (fxx < 0.04)
  cond_lf = (fxx >= 0.04) & (fxx < 0.15)
  cond_hf = (fxx >= 0.15) & (fxx < 0.4)
  
  # calculate power in each band by integrating the spectral density 
  vlf = trapz(pxx[cond_vlf], fxx[cond_vlf])
  lf = trapz(pxx[cond_lf], fxx[cond_lf])
  hf = trapz(pxx[cond_hf], fxx[cond_hf])
  
  # sum these up to get total power
  total_power = vlf + lf + hf

  # find which frequency has the most power in each band
  peak_vlf = fxx[cond_vlf][np.argmax(pxx[cond_vlf])]
  peak_lf = fxx[cond_lf][np.argmax(pxx[cond_lf])]
  peak_hf = fxx[cond_hf][np.argmax(pxx[cond_hf])]

  # fraction of lf and hf
  lf_nu = 100 * lf / (lf + hf)
  hf_nu = 100 * hf / (lf + hf)
  
  results = {}
  results['Power VLF (ms2)'] = vlf
  results['Power LF (ms2)'] = lf
  results['Power HF (ms2)'] = hf   
  results['Power Total (ms2)'] = total_power

  results['LF/HF'] = (lf/hf)
  results['Peak VLF (Hz)'] = peak_vlf
  results['Peak LF (Hz)'] = peak_lf
  results['Peak HF (Hz)'] = peak_hf

  results['Fraction LF (nu)'] = lf_nu
  results['Fraction HF (nu)'] = hf_nu
  
  # print("Frequency domain metrics:")
  # for k, v in results.items():
  #     print("- %s: %.2f" % (k, v))
  return results, fxx, pxx
def get_rr_corrected_ourdata(df):
  # detect peaks
  t = np.linspace(1.5 * np.pi, 3.5 * np.pi,13)
  qrs_filter = -np.sin(t)
  peaks, similarity = detect_peaks(df.heartrate, threshold=0.03,qrs_filter=qrs_filter)
 
  # group peaks so we get a single peak per beat (hopefully)
  grouped_peaks = group_peaks(peaks,threshold=3)
  # print(grouped_peaks)

  final=[element * 65 for element in grouped_peaks] 
  # print(final)
  # RR-intervals are the differences between successive peaks
  rr = np.diff(final)

  rr_corrected = rr.copy()
  rr_corrected[np.abs(zscore(rr)) > 2] = np.median(rr)
  return rr_corrected
def windowing(df,result_df,stress_level):
  df1=df.rolling(window=465)
  type(df1)
  i=1
  j=0
  for df2 in df1:
    if len(df2)>=464 and i%int(465/2)==0:
      # print((df2))
      results_td=[]
      results_fd=[]
      results=[]
      rr_corrected= get_rr_corrected_ourdata(df2)
      # plot_rr_corrected(rr_corrected)
      # print(rr_corrected)
      results_td=timedomain(rr_corrected)
      # print(results_td)
      results_fd, fxx, pxx =frequency_domain(rr_corrected)
      # print(results_fd)
      results={**results_td,**results_fd}
      results['stress_level']=stress_level
      # print(results)
      result_df=result_df.append(results,ignore_index=True)
      # print(result_df)
      i+=1
      j+=1
    else:
      i+=1
  return result_df


#Feature Extraction

Load DataSET

In [ ]:
df = pd.read_excel(os.path.join(data_path, "drive01.xlsx"))
df.ms=1000*df.ms                     #Convert senconds to MS

In [ ]:
column_names=['Mean RR (ms)', 'STD RR/SDNN (ms)', "Mean HR (Kubios' style) (beats/min)", 'Mean HR (beats/min)', 'STD HR (beats/min)', 'Min HR (beats/min)', 'Max HR (beats/min)', 'RMSSD (ms)', 'NNxx', 'pNNxx (%)', 'Power VLF (ms2)', 'Power LF (ms2)', 'Power HF (ms2)', 'Power Total (ms2)', 'LF/HF', 'Peak VLF (Hz)', 'Peak LF (Hz)', 'Peak HF (Hz)', 'Fraction LF (nu)', 'Fraction HF (nu)','stress_level']
result_df= pd.DataFrame(columns=column_names)


In [ ]:
ls_time=[600,900]           #Time in seconds for extraction of high, Low, MEdium stress features
ms_time=[3100,3400]
hs_time=[1700,2000]


df_low=df.iloc[int(ls_time[0]/0.0645):int(ls_time[1]/0.0645),:]
df_medium=df.iloc[int(ms_time[0]/0.0645):int(ms_time[1]/0.0645),:]
df_high=df.iloc[int(hs_time[0]/0.0645):int(hs_time[1]/0.0645),:]

In [ ]:
result_df=windowing(df_low,result_df,0)
result_df=windowing(df_medium,result_df,1)
result_df=windowing(df_high,result_df,2)
len(result_df)

/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 114, using nperseg = 114
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 113, using nperseg = 113
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 112, using nperseg = 112
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 116, using nperseg = 116
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral.py:1966: UserWarning: nperseg = 256 is greater than input length  = 117, using nperseg = 117
  .format(nperseg, input_length))
/usr/local/lib/python3.7/dist-packages/scipy/signal/spectral

57

In [ ]:
result_df

,Mean RR (ms),STD RR/SDNN (ms),Mean HR (Kubios' style) (beats/min),Mean HR (beats/min),STD HR (beats/min),Min HR (beats/min),Max HR (beats/min),RMSSD (ms),NNxx,pNNxx (%),Power VLF (ms2),Power LF (ms2),Power HF (ms2),Power Total (ms2),LF/HF,Peak VLF (Hz),Peak LF (Hz),Peak HF (Hz),Fraction LF (nu),Fraction HF (nu),stress_level
0,732.875000,71.185387,81.869350,82.645765,8.044368,68.376068,102.564103,105.376310,23.0,57.500000,101.197703,604.970119,2107.756209,2813.924031,0.287021,0.035088,0.140351,0.315789,22.301185,77.698815,0.0
1,726.375000,54.041940,82.601962,83.073316,6.356145,73.846154,97.165992,70.111518,20.0,50.000000,56.200659,690.805070,1274.120642,2021.126371,0.542182,0.035398,0.141593,0.283186,35.156803,64.843197,0.0
2,723.125000,53.279188,82.973207,83.437092,6.323683,71.005917,97.165992,70.014879,22.0,55.000000,34.208066,347.735063,2165.576718,2547.519847,0.160574,0.035714,0.142857,0.357143,13.835731,86.164269,0.0
3,710.357143,52.390962,84.464555,84.916775,6.156541,71.005917,97.165992,72.672209,20.0,47.619048,350.376474,239.738283,557.053085,1147.167842,0.430369,0.034483,0.068966,0.344828,30.087962,69.912038,0.0
4,664.034091,50.632970,90.356807,90.876543,6.849407,76.923077,108.597285,54.065798,15.0,34.090909,348.883116,109.960087,1319.201401,1778.044604,0.083354,0.035398,0.070796,0.283186,7.694028,92.305972,0.0
5,638.936170,53.329926,93.906094,94.558830,7.852193,80.267559,108.597285,59.465463,16.0,34.042553,140.446503,706.886655,616.694073,1464.027231,1.146252,0.034188,0.068376,0.273504,53.407143,46.592857,0.0
6,619.574468,55.852482,96.840659,97.618340,8.658711,83.916084,115.384615,57.502363,21.0,44.680851,45.121077,918.382682,764.837236,1728.340995,1.200756,0.035398,0.141593,0.176991,54.561063,45.438937,0.0
7,601.250000,94.372178,99.792100,101.687181,12.492622,57.692308,123.076923,132.248005,14.0,31.818182,114.253734,887.051254,10969.143633,11970.448621,0.080868,0.039216,0.117647,0.274510,7.481753,92.518247,0.0
8,778.088235,331.243235,77.112077,88.555366,27.815723,36.199095,123.076923,405.806576,24.0,70.588235,15958.317300,6707.027716,114475.450020,137140.795036,0.058589,0.039216,0.078431,0.274510,5.534651,94.465349,0.0
9,654.875000,210.442206,91.620538,97.548064,18.835386,43.956044,115.384615,196.866071,11.0,27.500000,10792.652076,3569.606520,6692.089816,21054.348412,0.533407,0.039604,0.079208,0.158416,34.785735,65.214265,0.0


In [ ]:
result_df.to_csv(os.path.join(output_path, "features.csv"))

'/content/drive/MyDrive/HRV/Github_Dataset/output/rr.txt'